In [22]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score, classification_report, confusion_matrix
import random

In [23]:
mannequin_path = os.path.join('../','mannequin_filter')
not_mannequin_path = os.path.join('../','not_mannequin_filter')

mannequin_img_path = [os.path.join(mannequin_path,x) for x in os.listdir(mannequin_path)]
not_mannequin_img_path = [os.path.join(not_mannequin_path,x) for x in os.listdir(not_mannequin_path)]

print(mannequin_img_path[:5])
print(not_mannequin_img_path[:5])

random.shuffle(mannequin_img_path)
random.shuffle(mannequin_img_path)
random.shuffle(not_mannequin_img_path)
random.shuffle(not_mannequin_img_path)

print(mannequin_img_path[:5])
print(not_mannequin_img_path[:5])

['../mannequin_filter/176.jpg', '../mannequin_filter/1234.jpg', '../mannequin_filter/229.jpg', '../mannequin_filter/112.jpg', '../mannequin_filter/457.jpg']
['../not_mannequin_filter/176.jpg', '../not_mannequin_filter/1234.jpg', '../not_mannequin_filter/229.jpg', '../not_mannequin_filter/112.jpg', '../not_mannequin_filter/457.jpg']
['../mannequin_filter/1208.jpg', '../mannequin_filter/299.jpg', '../mannequin_filter/760.jpg', '../mannequin_filter/993.jpg', '../mannequin_filter/1046.jpg']
['../not_mannequin_filter/460.jpg', '../not_mannequin_filter/229.jpg', '../not_mannequin_filter/639.jpg', '../not_mannequin_filter/374.jpg', '../not_mannequin_filter/389.jpg']


In [24]:
train_mannequin_img, test_mannequin_img = train_test_split(mannequin_img_path,test_size = 0.25,random_state=42)
train_mannequin_img, val_mannequin_img = train_test_split(train_mannequin_img,test_size = 0.25,random_state=42)


In [25]:
test_not_mannequin_img,val_not_mannequin_img = train_test_split(not_mannequin_img_path,test_size = 0.25, random_state=42)

In [26]:
def generate_label_list(path,test_img_path):
    
    label_list = []
    for x in path:
        if x in test_img_path:
            label_list.append(1)
        else:
            label_list.append(0)
    return label_list

In [27]:
all_test_paths = test_mannequin_img + test_not_mannequin_img
random.shuffle(all_test_paths)
test_label_list = generate_label_list(all_test_paths,test_mannequin_img)

all_val_paths = val_mannequin_img + val_not_mannequin_img
random.shuffle(all_val_paths)
val_label_list = generate_label_list(all_val_paths,val_mannequin_img)


In [28]:
print(test_label_list[-8:])
print(all_test_paths[-8:])

[0, 0, 1, 0, 1, 0, 0, 0]
['../not_mannequin_filter/573.jpg', '../not_mannequin_filter/618.jpg', '../mannequin_filter/462.jpg', '../not_mannequin_filter/324.jpg', '../mannequin_filter/343.jpg', '../not_mannequin_filter/397.jpg', '../not_mannequin_filter/543.jpg', '../not_mannequin_filter/335.jpg']


In [29]:
image_size = 224
def read_and_prep_images(img_paths, img_height=image_size, img_width=image_size):
    imgs = [load_img(img_path, target_size=(img_height, img_width)) for img_path in img_paths]
    img_array = np.array([img_to_array(img) for img in imgs])
    #output = img_array
    output = preprocess_input(img_array)
    return(output)


In [30]:
X_train = read_and_prep_images(train_mannequin_img)
X_test = read_and_prep_images(all_test_paths)
X_val = read_and_prep_images(all_val_paths)

In [31]:
resnet_model = ResNet50(include_top = False, weights='imagenet',input_shape=(224,224,3))
resnet_model.trainable = False
x = tf.keras.layers.GlobalAveragePooling2D()(resnet_model.output)
x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.Flatten()(resnet_model.output)
model = tf.keras.Model(resnet_model.input,x)
# model = resnet_model

In [32]:
print(model.summary())

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [33]:
X_train = model.predict(X_train)
X_test = model.predict(X_test)
X_val = model.predict(X_val)


In [34]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn import svm

In [35]:
ss = StandardScaler()
ss.fit(X_train)
X_train = ss.transform(X_train)
X_test = ss.transform(X_test)
X_val = ss.transform(X_val)

# Take PCA to reduce feature space dimensionality
pca = PCA(n_components=512, whiten=True)
pca = pca.fit(X_train)
print('Explained variance percentage = %0.2f' % sum(pca.explained_variance_ratio_))
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)
X_val = pca.transform(X_val)

Explained variance percentage = 0.99


In [36]:
# Train classifier and obtain predictions for OC-SVM
oc_svm_clf = svm.OneClassSVM(gamma=0.00001, kernel='rbf', nu=0.08)  # Obtained using grid search
if_clf = IsolationForest(contamination=0.08, max_features=1.0, max_samples=1.0, n_estimators=40)  # Obtained using grid search

oc_svm_clf.fit(X_train)
if_clf.fit(X_train)

oc_svm_preds = oc_svm_clf.predict(X_test)
if_preds = if_clf.predict(X_test)

In [37]:
oc_svm_preds


array([-1, -1, -1, ..., -1, -1, -1])

In [38]:
if_preds[0:10]

array([-1, -1, -1, -1,  1,  1, -1, -1, -1, -1])

In [39]:
if_preds_modified = []
for i in range(len(if_preds)):
    if if_preds[i] == -1:
        if_preds_modified.append(0)
        
    else:
        if_preds_modified.append(1)

oc_svm_pred_modified = []
for i in range(len(oc_svm_preds)):
    if oc_svm_preds[i] == -1:
        oc_svm_pred_modified.append(0)
        
    else:
        oc_svm_pred_modified.append(1)


In [40]:
print(roc_auc_score(test_label_list,if_preds_modified))

0.8589743589743589


In [41]:
print(roc_auc_score(test_label_list,oc_svm_pred_modified))

0.8824786324786326


In [42]:
print(oc_svm_clf.score_samples(X_test))

[54.38698683 54.66305655 55.04232816 ... 53.75560598 55.08281754
 55.09551496]
